# Plot of SST/HF radar data for S-MODE

Make a plot of SST with HF radar current vectors overlaid.  This notebook is branched off from "HF_radat_plot.ipynb".  Changes to do:
- Would like to make is so that a single date can be entered to select the SST image and coincident HF radar data.
- Inputs: date, lat/lon domain, amount of averaging time, colorscale, ??
- Should I do this with flags to select cases (e.g., case_flag=1 selects a certain set of input parameters, like time, lat/lon domain, colorscale)?


Other notes:
- Uses data from [HFRnet THREDDS server](http://hfrnet-tds.ucsd.edu/thredds/catalog.html)
- Draws from https://github.com/rowg/HFRnet-Thredds-support/blob/master/PythonNotebooks/TimeseriesRTVfromSIO_TDS.ipynb


## Notes on situations and good SST images

Good one well before start of experiment, 2021-09-29
```url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20210929T213000Z.nc'  #This one is good```

2021-10-16, good one at start of experiment, 
```#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211016T095000Z.nc' # good one```

2021-10-19, 03:10 Metop A, Really good one near start of experiment:
``` # url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/AVHRR_METOPA/AVHRR_METOPA_20211019T031000Z.nc#bytes' #Really great! ```

Not awesome, but coincident with B200 flight on 10/19,
```# url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/VIIRS_NRT/VIIRS_NRT_20211019T103001Z.nc#mode=bytes' ```

Not sure about these-- I should check them:
```
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20211011T233459Z.nc'
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211022T111000Z.nc'
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPA/AVHRR_METOPA_20211022T161000Z.nc' # good one!
```

Two of the best, from 2021-10-19 and 2021-10-25 that make a good sequence of the early part of the experiment
```
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPC/AVHRR_METOPC_20211019T052000Z.nc'  # prettiest of all!!
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211025T214001Z.nc'
```

Really good ones from last two days of the campaing, 11-4 and 11-5:
```
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211104T103001Z.nc'
url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211105T101000Z.nc'
```

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime
import copy

import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
import functions  # requires functions.py from this repository
import datetime

import sys
sys.path.insert(0,'../../NASA_SMODE/DataSynthesis/tools')
sys.path.insert(0,'../../NASA_SMODE/DataSynthesis/data-synthesis/')
from mapping_tools import *
from tools import *

In [2]:
plt.rcParams['figure.figsize'] = (8,8)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 700
plt.close('all')

# %matplotlib inline
%matplotlib qt5

__figdir__ = '../plots/' 
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
f = gsw.geostrophy.f(37)
inertial_period = 2*np.pi/f/3600
print('Inertial period is '+ f'{inertial_period:.1f}' + ' hours')

Inertial period is 19.9 hours


In [4]:
# Good one well before start of experiment, 2021-09-29
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20210929T213000Z.nc'  #This one is good

# 2021-10-16, good one at start of experiment, 
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211016T095000Z.nc' # good one

# 2021-10-19, 03:10 Metop A, Really good one near start of experiment:
# url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/AVHRR_METOPA/AVHRR_METOPA_20211019T031000Z.nc#bytes' #Really great! 

# Not awesome, but coincident with B200 flight on 10/19,
# url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/VIIRS_NRT/VIIRS_NRT_20211019T103001Z.nc#mode=bytes'

# Not sure about these-- I should check them:
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20211011T233459Z.nc'
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211022T111000Z.nc'

# Good sequence from 2021-10-19, 2021-10-22 and 2021-10-25 showing early part of the experiment
#  (Western part of domain-- not great for HF radar)
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPC/AVHRR_METOPC_20211019T052000Z.nc'; V = [12, 14.5]; zoom = 1  # prettiest of all!!
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPA/AVHRR_METOPA_20211022T161000Z.nc'; V = [12, 15.5]; zoom = 1 # good one!
# potentially useful, not great: url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211022T111000Z.nc'; V = [12, 15.5]
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211025T214001Z.nc'; V = [12, 15.5]; zoom = 1

# Phase 2 of experiment (2021-10-28):
# This one is really cool with the HF radar
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPA/AVHRR_METOPA_20211028T033000Z.nc'; V = [14, 16]; zoom = 1

# Really good ones from last two days of the campaign, 11-4 and 11-5:
url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211104T103001Z.nc'; V = [14, 16]; zoom = 3 #very good if zoomed on study region
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211105T101000Z.nc'; V = [14, 16]; zoom = 3 #not totally great, but pretty good, especially if zoomed



In [5]:
time_window = 20
zoom_str = 'zoom' + str(zoom)

In [6]:
savefig = True

In [7]:
(ax,startTimeDT,endTimeDT,day_str2) = functions.sst_map_SMODE(url,zoom,V,time_window)

C:\Users\jtomf\anaconda3\envs\base2\lib\site-packages\cartopy\mpl\gridliner.py:343: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


{'transform': <cartopy.crs.PlateCarree object at 0x0000027E44475040>, 'color': 'w'}


In [8]:

'''# Add a 10 km scale bar
km_per_deg_lat=gsw.geostrophy.distance((125,125), (37,38))/1000
deg_lat_equal_10km=10/km_per_deg_lat
x0 = -125
y0 = 37.75
ax.plot(x0+np.asarray([0, 0]),y0+np.asarray([0.,deg_lat_equal_10km]),transform=ccrs.PlateCarree(),color='k')
ax.text(x0+2/60, y0-.5/60, '10 km', fontsize=6,transform=ccrs.PlateCarree())
''';

In [9]:
'''saildrones=functions.get_current_position('saildrone')#,'ko', markersize=3,transform=ccrs.PlateCarree())
hSD = ax.plot(saildrones['longitude'],saildrones['latitude'],'wo',markersize=5,transform=ccrs.PlateCarree())
wg=functions.get_current_position('waveglider')#,'ko', markersize=3,transform=ccrs.PlateCarree())
hWG = ax.plot(wg['longitude'],wg['latitude'],'mo',markersize=5,transform=ccrs.PlateCarree())''';

In [10]:
if savefig:
    plt.savefig(__figdir__+'SST_UV_' + day_str2 + zoom_str + '_' + f'{time_window:.0f}' + 'hr' + '.' +plotfiletype,**savefig_args)

In [11]:
plt.rcParams['savefig.dpi']

700.0

In [12]:
plt.rcParams['figure.figsize'] 

[6.0, 4.0]

# Now read in locations of in situ platforms

In [13]:
BASE_URL = 'http://smode.whoi.edu:8080/thredds/dodsC/insitu/'

saildrones = {
    'type': 'sd',
    'ids': ['1072','1073','1062','1075','1074'],
    'URL': BASE_URL + 'saildrone/',
    'rename_database' : {
    'sbe37_practical_salinity_filtered': 'salinity',
    'sbe37_temperature_filtered': 'temperature',
    'water_velocity_east': 'u',
    'water_velocity_north': 'v',
    },
}

wavegliders = {
    'type': 'wg',
    'ids': ['KELVIN','STOKES','SV3'],
    'URL': BASE_URL + 'waveglider/',
    'rename_database': {
        'longitude_1hz_Avg': 'longitude',
        'latitude_1hz_Avg': 'latitude',
        'uctd_temp_Avg':  'temperature',
        'uctd_cond_Avg': 'conductivity',
    }
}

oceanus = {
    'type': 'rv',
    'ids': ['oceanus'],
    'URL': BASE_URL + 'Oceanus/',
    'rename_database':{
        'tsg_flth_temperature': 'temperature',
        'tsg_flth_salinity': 'salinity',
        'gnss_gga_bow_longitude': 'longitude',
        'gnss_gga_bow_latitude': 'latitude',
    }
}

In [14]:
# oceanus
filename = 'cur_obs.nc'
ds = xr.open_dataset(oceanus['URL'] + filename).sel(time=slice(startTimeDT,endTimeDT))
ds = ds.rename(oceanus['rename_database'])
ds = ds.assign_coords({'longitude': ds.longitude,
                       'latitude': ds.latitude
                      }
)
rv_oceanus = ds

In [15]:
startTimeDT.strftime("%Y-%m-%d")

'2021-11-04'

In [16]:
# wavegliders
for wg in ['KELVIN']:
    
    if wg == 'SV3':
#         !wget http://smode.whoi.edu:8080/thredds/catalog/insitu/waveglider/catalog.html?dataset=insitu/waveglider/SV3-1043_PLD2_TAB1.nc
        filename =  wg + '-1043_PLD2_TAB1.nc'
        ds = xr.open_dataset(wavegliders['URL'] + filename).sel(time=slice(startTimeDT,endTimeDT))

    else:
        filename = wg + '_PLD2_TAB1.nc'
        ds = xr.open_dataset(wavegliders['URL'] + filename).sel(time=slice(startTimeDT,endTimeDT))
    
    
    ds = ds.rename(wavegliders['rename_database'])
    ds = ds.assign_coords({'longitude': ds.longitude,
                           'latitude': ds.latitude
                          }
    )
    
    ds['salinity'] = gsw.SP_from_C(ds['conductivity']*10, # S/m to mS/cm
                                   ds['temperature'],p=2)
    
    exec('wg_'+wg+" = ds")

In [17]:
wg

'KELVIN'

In [18]:
ax.plot(rv_oceanus.longitude, rv_oceanus.latitude,'k', transform=ccrs.PlateCarree())

for wg in ['KELVIN']:
    exec('ds = '+'wg_' + wg)
    ax.plot(ds.longitude, ds.latitude,'m', transform=ccrs.PlateCarree())

'''for sd in saildrones['ids']:
    exec('ds = ' + 'sd_' + sd)
    ax.plot(ds.longitude, ds.latitude,'y', transform=ccrs.PlateCarree())
'''
h_Oceanus = matplotlib.patches.Patch(color='k', label='The red data')
h_wg = matplotlib.patches.Patch(color='m', label='The red data')
ax.legend([h_Oceanus,h_wg],['RV $\it{Oceanus}$', 'Wave Gliders'], loc ='lower left')

In [19]:
if savefig:
    plt.savefig(__figdir__+'SST_UV_' + day_str2 + zoom_str + '_' + f'{time_window:.0f}' + 'hr' + '.' +plotfiletype,**savefig_args)